# Import Libraries


In [25]:
import cv2
import numpy as np


# Enter video path

In [26]:
video_path = 'video.mp4'

# Function to detect and track balls in a video

In [41]:


def detect_and_track_balls(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Unable to open video file.")
        return
    
    # Get video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)  # Obtain FPS here
    
    # Check if FPS is valid
    if fps == 0:
        print("Error: FPS could not be determined.")
        cap.release()
        return
    
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Define quadrants
    q1_x1, q1_y1 = 0, 0
    q1_x2, q1_y2 = width // 2, height // 2  # Quadrant 1
    
    q2_x1, q2_y1 = width // 2, 0
    q2_x2, q2_y2 = width, height // 2  # Quadrant 2
    
    q3_x1, q3_y1 = 0, height // 2
    q3_x2, q3_y2 = width // 2, height  # Quadrant 3
    
    q4_x1, q4_y1 = width // 2, height // 2
    q4_x2, q4_y2 = width, height  # Quadrant 4
    
    quadrants = [
        (q1_x1, q1_y1, q1_x2, q1_y2),  # Quadrant 1
        (q2_x1, q2_y1, q2_x2, q2_y2),  # Quadrant 2
        (q3_x1, q3_y1, q3_x2, q3_y2),  # Quadrant 3
        (q4_x1, q4_y1, q4_x2, q4_y2)   # Quadrant 4
    ]
    
    # Dictionary to store ball positions
    ball_positions = {}
    
    # List to store event logs
    event_log = []
    
    frame_number = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Convert frame to HSV color space
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        # Define color ranges (example for red, blue, and green)
        colors = {
            "red": ([0, 120, 70], [10, 255, 255]),
            "blue": ([94, 80, 2], [126, 255, 255]),
            "green": ([25, 52, 72], [102, 255, 255])
        }
        
        for color, (lower, upper) in colors.items():
            # Create mask for color
            mask = cv2.inRange(hsv, np.array(lower), np.array(upper))
            contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            for contour in contours:
                area = cv2.contourArea(contour)
                if area > 500:
                    # Get bounding box for contour
                    x, y, w, h = cv2.boundingRect(contour)
                    cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                    
                    # Determine quadrant
                    cx, cy = x + w // 2, y + h // 2
                    for i, (qx1, qy1, qx2, qy2) in enumerate(quadrants):
                        if qx1 <= cx < qx2 and qy1 <= cy < qy2:
                            quadrant = i + 1
                            break
                    
                    # Log entry or exit events
                    if color not in ball_positions:
                        ball_positions[color] = (cx, cy, quadrant)
                        event_type = "Entry"
                    else:
                        prev_cx, prev_cy, prev_quadrant = ball_positions[color]
                        if prev_quadrant != quadrant:
                            event_type = "Exit" if prev_cx < qx1 or prev_cx >= qx2 or prev_cy < qy1 or prev_cy >= qy2 else "Entry"
                        else:
                            event_type = None
                    
                    if event_type:
                        timestamp = frame_number / fps
                        event_log.append((timestamp, quadrant, color, event_type))
                        cv2.putText(frame, f"{event_type} {quadrant}", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)
                    
                    # Update ball position
                    ball_positions[color] = (cx, cy, quadrant)
        
        # Display frame
        cv2.imshow('Frame', frame)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
        
        frame_number += 1
    
    # Release video capture and destroy windows
    cap.release()
    cv2.destroyAllWindows()
    
    # Write events to text file
    with open("event_log.txt", "w") as file:
        for event in event_log:
            file.write(f"{event[0]:.2f}, {event[1]}, {event[2]}, {event[3]}\n")

# Specify path to video file
video_path = 'video.mp4'

# Call function to detect and track balls
detect_and_track_balls(video_path)
